In [1]:
import pulp
import pandas as pd
import io
import itertools

In [2]:
# 輸送問題

In [3]:
supply = {1:100, 2:180}
demand = {1:120, 2:40, 3:80}

COSTS = io.StringIO("""
S,D,C
1,1,10
1,2,6
1,3,16
2,1,8
2,2,8
2,3,10
""")
df_costs = pd.read_csv(COSTS)

In [4]:
df_costs['Var'] = [pulp.LpVariable(f'x{i}{j}', 0) for i, j in itertools.product(supply.keys(), demand.keys())]

In [5]:
df_costs

,S,D,C,Var
0,1,1,10,x11
1,1,2,6,x12
2,1,3,16,x13
3,2,1,8,x21
4,2,2,8,x22
5,2,3,10,x23


In [6]:
pl = pulp.LpProblem('problem_logistics')
pl += pulp.lpDot(df_costs.C, df_costs.Var), 'objective_function'
for i, v in df_costs.groupby('S'):
    pl += pulp.lpSum(v.Var) <= supply[i]
for j, v in df_costs.groupby('D'):
    pl += pulp.lpSum(v.Var) == demand[j]
pl

problem_logistics:
MINIMIZE
10*x11 + 6*x12 + 16*x13 + 8*x21 + 8*x22 + 10*x23 + 0
SUBJECT TO
_C1: x11 + x12 + x13 <= 100

_C2: x21 + x22 + x23 <= 180

_C3: x11 + x21 = 120

_C4: x12 + x22 = 40

_C5: x13 + x23 = 80

VARIABLES
x11 Continuous
x12 Continuous
x13 Continuous
x21 Continuous
x22 Continuous
x23 Continuous

In [7]:
result = pl.solve()
pulp.LpStatus[result]

'Optimal'

In [8]:
pulp.value(pl.objective)

2040.0

In [9]:
for v in pl.variables():
    print(f"{v} = {pulp.value(v)}")

x11 = 20.0
x12 = 40.0
x13 = 0.0
x21 = 100.0
x22 = 0.0
x23 = 80.0


In [10]:
# 割り当て問題

In [11]:
CSV = io.StringIO("""
Actor,Role,Score
1,1,3
1,2,6
1,3,9
1,4,8
2,1,6
2,2,3
2,3,2
2,4,4
3,1,9
3,2,3
3,3,2
3,4,5
4,1,6
4,2,2
4,3,3
4,4,8
""")
df_roles = pd.read_csv(CSV)

In [12]:
df_roles['Var'] = [pulp.LpVariable(f'x{i + 1}{j + 1}', cat = pulp.LpBinary) for i, j in itertools.product(range(4), range(4))]

In [13]:
pa = pulp.LpProblem('problem_assign', sense = pulp.LpMaximize)
# Varには0-1で値が入るため、スコアとVarの内積＝スコアの総和になる
pa += pulp.lpDot(df_roles.Score, df_roles.Var), 'objective_fucntion'
for _, v in df_roles.groupby('Actor'):
    pa += pulp.lpSum(v.Var) == 1
for _, v in df_roles.groupby('Role'):
    pa += pulp.lpSum(v.Var) == 1
pa

problem_assign:
MAXIMIZE
3*x11 + 6*x12 + 9*x13 + 8*x14 + 6*x21 + 3*x22 + 2*x23 + 4*x24 + 9*x31 + 3*x32 + 2*x33 + 5*x34 + 6*x41 + 2*x42 + 3*x43 + 8*x44 + 0
SUBJECT TO
_C1: x11 + x12 + x13 + x14 = 1

_C2: x21 + x22 + x23 + x24 = 1

_C3: x31 + x32 + x33 + x34 = 1

_C4: x41 + x42 + x43 + x44 = 1

_C5: x11 + x21 + x31 + x41 = 1

_C6: x12 + x22 + x32 + x42 = 1

_C7: x13 + x23 + x33 + x43 = 1

_C8: x14 + x24 + x34 + x44 = 1

VARIABLES
0 <= x11 <= 1 Integer
0 <= x12 <= 1 Integer
0 <= x13 <= 1 Integer
0 <= x14 <= 1 Integer
0 <= x21 <= 1 Integer
0 <= x22 <= 1 Integer
0 <= x23 <= 1 Integer
0 <= x24 <= 1 Integer
0 <= x31 <= 1 Integer
0 <= x32 <= 1 Integer
0 <= x33 <= 1 Integer
0 <= x34 <= 1 Integer
0 <= x41 <= 1 Integer
0 <= x42 <= 1 Integer
0 <= x43 <= 1 Integer
0 <= x44 <= 1 Integer

In [14]:
result = pa.solve()
pulp.LpStatus[result]

'Optimal'

In [15]:
 pulp.value(pa.objective)

29

In [16]:
for v in pa.variables():
    print(f"{v} = {pulp.value(v)}")

x11 = 0
x12 = 0
x13 = 1
x14 = 0
x21 = 0
x22 = 1
x23 = 0
x24 = 0
x31 = 1
x32 = 0
x33 = 0
x34 = 0
x41 = 0
x42 = 0
x43 = 0
x44 = 1


In [17]:
df_roles['Val'] = df_roles.Var.apply(pulp.value)
df_roles[df_roles.Val > 0]

,Actor,Role,Score,Var,Val
2,1,3,9,x13,1
5,2,2,3,x22,1
8,3,1,9,x31,1
15,4,4,8,x44,1


In [18]:
# ネットワーク

In [19]:
CSV = io.StringIO("""
i,j,W
1,2,1
1,3,5
2,1,1
2,3,2
2,4,2
3,1,5
3,2,4
3,5,2
4,3,3
4,6,3
5,3,1
5,6,4
""")
df_net = pd.read_csv(CSV)

In [20]:
df_net['Var'] = [pulp.LpVariable(f"x{df_net.i[t]}{df_net.j[t]}", cat = pulp.LpBinary) for t in df_net.index]
df_net['Var']

0     x12
1     x13
2     x21
3     x23
4     x24
5     x31
6     x32
7     x35
8     x43
9     x46
10    x53
11    x56
Name: Var, dtype: object

In [21]:
pn = pulp.LpProblem('problem_network', sense = pulp.LpMinimize)
pn += pulp.lpDot(df_net.W, df_net.Var), 'objective_function'
begin = 1
end = 5
pn += pulp.lpSum(df_net.Var[df_net.i == begin]) - pulp.lpSum(df_net.Var[df_net.j == begin]) == 1, 'start_node'
pn += pulp.lpSum(df_net.Var[df_net.i == end]) - pulp.lpSum(df_net.Var[df_net.j == end]) == -1, 'end_node'
for i in set(df_net.i) | set(df_net.j):
    if(i != begin and i != end):
        pn += pulp.lpSum(df_net.Var[df_net.i == i]) - pulp.lpSum(df_net.Var[df_net.j == i]) == 0, f'route_node_{i}'
pn

problem_network:
MINIMIZE
1*x12 + 5*x13 + 1*x21 + 2*x23 + 2*x24 + 5*x31 + 4*x32 + 2*x35 + 3*x43 + 3*x46 + 1*x53 + 4*x56 + 0
SUBJECT TO
start_node: x12 + x13 - x21 - x31 = 1

end_node: - x35 + x53 + x56 = -1

route_node_2: - x12 + x21 + x23 + x24 - x32 = 0

route_node_3: - x13 - x23 + x31 + x32 + x35 - x43 - x53 = 0

route_node_4: - x24 + x43 + x46 = 0

route_node_6: - x46 - x56 = 0

VARIABLES
0 <= x12 <= 1 Integer
0 <= x13 <= 1 Integer
0 <= x21 <= 1 Integer
0 <= x23 <= 1 Integer
0 <= x24 <= 1 Integer
0 <= x31 <= 1 Integer
0 <= x32 <= 1 Integer
0 <= x35 <= 1 Integer
0 <= x43 <= 1 Integer
0 <= x46 <= 1 Integer
0 <= x53 <= 1 Integer
0 <= x56 <= 1 Integer

In [22]:
result = pn.solve()
pulp.LpStatus[result]

'Optimal'

In [23]:
pulp.value(pn.objective)

5

In [24]:
df_net[df_net.Var.apply(pulp.value) > 0]

,i,j,W,Var
0,1,2,1,x12
3,2,3,2,x23
7,3,5,2,x35


In [25]:
# 最大流問題

In [26]:
CSV = io.StringIO("""
i,j,U
1,2,4
1,3,3
2,1,1
2,3,4
2,5,3
3,4,3
3,5,2
4,5,2
5,3,2
""")
df_r = pd.read_csv(CSV)

In [27]:
# 決定変数は容量制約に従う
df_r['Var'] = [pulp.LpVariable(f"x{df_r.i[i]}{df_r.j[i]}", lowBound=0, upBound=df_r.U[i]) for i in df_r.index]
z = pulp.LpVariable("z")

In [28]:
pr = pulp.LpProblem('problem_route_maximize_flow', sense = pulp.LpMaximize)
pr += z, 'objective_function_sum_of_flow' # Z=始点からの送出量の最大化
start_node = 1
end_node = 5
send_nodes = set(df_r.i)
receive_nodes = set(df_r.j)
for node in send_nodes | receive_nodes: # 各ノードの流量制約
    if node == start_node:
        pr += pulp.lpSum(df_r.Var[df_r.i == node]) - pulp.lpSum(df_r.Var[df_r.j == node]) == z # 開始ノードからは総送出量ぶん出ていく
    elif node == end_node:
        pr += pulp.lpSum(df_r.Var[df_r.i == node]) - pulp.lpSum(df_r.Var[df_r.j == node]) == -z # 終点ノードへは総送出量ぶん入っていく
    else:
        pr += pulp.lpSum(df_r.Var[df_r.i == node]) - pulp.lpSum(df_r.Var[df_r.j == node]) == 0
pr

problem_route_maximize_flow:
MAXIMIZE
1*z + 0
SUBJECT TO
_C1: x12 + x13 - x21 - z = 0

_C2: - x12 + x21 + x23 + x25 = 0

_C3: - x13 - x23 + x34 + x35 - x53 = 0

_C4: - x34 + x45 = 0

_C5: - x25 - x35 - x45 + x53 + z = 0

VARIABLES
x12 <= 4 Continuous
x13 <= 3 Continuous
x21 <= 1 Continuous
x23 <= 4 Continuous
x25 <= 3 Continuous
x34 <= 3 Continuous
x35 <= 2 Continuous
x45 <= 2 Continuous
x53 <= 2 Continuous
z free Continuous

In [29]:
result = pr.solve()
pulp.LpStatus[result]

'Optimal'

In [30]:
pulp.value(pr.objective)

7.0

In [31]:
df_r['Val'] = df_r.Var.apply(pulp.value)
df_r

,i,j,U,Var,Val
0,1,2,4,x12,4.0
1,1,3,3,x13,3.0
2,2,1,1,x21,0.0
3,2,3,4,x23,1.0
4,2,5,3,x25,3.0
5,3,4,3,x34,2.0
6,3,5,2,x35,2.0
7,4,5,2,x45,2.0
8,5,3,2,x53,0.0


In [32]:
# 最小費用流問題

In [33]:
CSV = io.StringIO("""
i,j,C,U
1,2,20,30
1,3,10,5
2,3,5,10
2,4,10,20
3,4,20,20
3,5,10,5
4,5,20,10
""")
df_c = pd.read_csv(CSV)
df_c

,i,j,C,U
0,1,2,20,30
1,1,3,10,5
2,2,3,5,10
3,2,4,10,20
4,3,4,20,20
5,3,5,10,5
6,4,5,20,10


In [34]:
# 決定変数の容量制約
df_c['Var'] = [pulp.LpVariable(f"x{df_c.i[i]}{df_c.j[i]}", lowBound=0, upBound=df_c.U[i]) for i in df_c.index]
df_c['Var']

0    x12
1    x13
2    x23
3    x24
4    x34
5    x35
6    x45
Name: Var, dtype: object

In [35]:
pc = pulp.LpProblem('problem_route_minimize_cost', sense = pulp.LpMinimize)
pc += pulp.lpDot(df_c.Var, df_c.C), 'objective_function_total_cost'
B = {1:30, 2:0, 3:0, 4:-20, 5:-10} # 各ノードの正味供給量
nodes = B.keys()
for node in nodes:
    pc += pulp.lpSum(df_c.Var[df_c.i == node]) - pulp.lpSum(df_c.Var[df_c.j == node]) == B[node] # 各ノードの出入りは、正味供給量に等しい
pc

problem_route_minimize_cost:
MINIMIZE
20*x12 + 10*x13 + 5*x23 + 10*x24 + 20*x34 + 10*x35 + 20*x45 + 0
SUBJECT TO
_C1: x12 + x13 = 30

_C2: - x12 + x23 + x24 = 0

_C3: - x13 - x23 + x34 + x35 = 0

_C4: - x24 - x34 + x45 = -20

_C5: - x35 - x45 = -10

VARIABLES
x12 <= 30 Continuous
x13 <= 5 Continuous
x23 <= 10 Continuous
x24 <= 20 Continuous
x34 <= 20 Continuous
x35 <= 5 Continuous
x45 <= 10 Continuous

In [36]:
result = pc.solve()
pulp.LpStatus[result]

'Optimal'

In [37]:
pulp.value(pc.objective)

1025.0

In [38]:
df_c['Val'] = df_c.Var.apply(pulp.value)
df_c

,i,j,C,U,Var,Val
0,1,2,20,30,x12,25.0
1,1,3,10,5,x13,5.0
2,2,3,5,10,x23,5.0
3,2,4,10,20,x24,20.0
4,3,4,20,20,x34,5.0
5,3,5,10,5,x35,5.0
6,4,5,20,10,x45,5.0


In [39]:
# 最小値

In [65]:
CSV = io.StringIO("""
g,a,e,c
5,4,0,3
2,0,0,3
0,5,4,5
5,4,4,3
""")
df = pd.read_csv(CSV)
df

,g,a,e,c
0,5,4,0,3
1,2,0,0,3
2,0,5,4,5
3,5,4,4,3


In [137]:
vars = [pulp.LpVariable(f"x{i}-{j}", cat = pulp.LpBinary) for i in df.index for j in df.columns]
t = pulp.LpVariable(f"t")
columns = len(df.columns)
rows = len(df.index)

g    2
a    0
e    0
c    3
Name: 1, dtype: int64

In [140]:
p = pulp.LpProblem("min", sense = pulp.LpMaximize)
p += t, "objective function"
for i in range(rows):
    p += pulp.lpSum([vars[i*columns + j] for j in range(columns) if df.iloc[i, j] != 0]) == 1
for j in range(columns):
    p += pulp.lpSum([vars[i*columns + j] for i in range(rows) if df.iloc[i, j] != 0]) == 1
for i in range(rows):
    p += t <= pulp.lpDot(vars[i*columns:i*columns+columns], df.iloc[i, :])

In [142]:
result = p.solve()
pulp.LpStatus[result]

'Optimal'

In [149]:
for v in vars:
    print(pulp.value(v))

1
0
None
0
0
None
None
1
None
0
1
0
0
1
0
0
